In [24]:
from ecdsa import SigningKey, SECP256k1, VerifyingKey, BadSignatureError
import hashlib
import base64
import binascii
from datetime import datetime

class Wallet:
    """
        钱包
    """
    def __init__(self):
        """
            钱包初始化时基于椭圆曲线生成一个唯一的秘匙对，代表区块链上一个唯一的账户
        """
        self._private_key = SigningKey.generate(curve=SECP256k1)
        self._public_key = self._private_key.get_verifying_key()
    
    @property
    def address(self):
        """
            这里通过公匙生成地址
        """
        h = hashlib.sha256(self._public_key.to_pem())
        return base64.b64encode(h.digest())
    
    @property
    def pubkey(self):
        """
            返回公匙字符
        """
        return self._public_key.to_pem()
    
    def sign(self, message):
        """
            生成数字签名
        """
        h = hashlib.sha256(message.encode('utf8'))
        return binascii.hexlify(self._private_key.sign(h.digest()))
    

In [25]:
def verify_sign(pubkey, message, signature):
    """
        验证签名
    """
    verifier = VerifyingKey.from_pem(pubkey)
    h = hashlib.sha256(message.encode('utf8'))
    return verifier.verify(binascii.unhexlify(signature), h.digest())

In [26]:
#新建钱包

w = Wallet()

w.address

b'0WJ2SzW6BxCXmTNL9rP6w2MiOvFzTj7/PDpqeHVPE6I='

In [27]:
w.pubkey

b'-----BEGIN PUBLIC KEY-----\nMFYwEAYHKoZIzj0CAQYFK4EEAAoDQgAEudyis6N/3u5Zt+h/jwLv1zLV3CD1hxC+\nuckgQVkkSX+oXVIaks6gVWPQBbVECzYIR8lEiB0EEoyI8GIeOJU5QA==\n-----END PUBLIC KEY-----\n'

In [28]:
data="交易数据"

In [29]:
sig = w.sign(data)

print(sig)

b'f63bb664144677b35f29814f17fffbfb4cf7f6bc37e3126f2f58e2471d67449ca4570d8ad3a42abd15080b533dc8389d54db02e6cbb11f97dc9797f8e5c94720'


In [30]:
verify_sign(w.pubkey, data, sig)

True

In [31]:
import json
from json import JSONEncoder

class Transaction:
    """
        交易的结构
    """
    
    def __init__(self, sender, recipient, amount):
        """
            初始化交易，设置交易的发送方，接收方，和交易数量
        """
        if isinstance(sender, bytes):
            sender = sender.decode('utf-8')
        self.sender = sender
        if isinstance(recipient, bytes):
            recipient = recipient.decode('utf-8')
        self.recipient = recipient
        self.amount = amount
    
    def set_sign(self, signature, pubkey):
        """
            为了便于验证交易的可靠性，需要发送方输入他的公匙和签名
        """
        self.signature = signature
        self.pubkey = pubkey
    
    def __repr__(self):
        """
            交易大致可分为两种，一种是挖矿所得，二是转账交易
            挖矿物发送方，以此进行区分现实不同内容
        """
        if self.sender:
            s = "%s sending %d number of coins to %s"%(self.sender, self.amount, self.recipient)
        else:
            s = "%s getting %d number of coins from mining"%(self.recipient, self.amount)
        return s

In [32]:
#updated blockchain version with transactions:

class Block:
    """
        区块结构
            prev_hash:  父区块哈希值
            transactions:交易内容
            time stamp: 区块创建时间
            hash:       区块哈希值
            Nonce       随机数
    """
    
    def __init__(self, transactions, prev_hash):
        self.prev_hash = prev_hash
        self.transactions = transactions
        self.timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        #不计算区块的哈希值，设置初始Nonce和哈希为None
        self.nonce = None
        self.hash = None
        
    def __repr__(self):
        return"transactions: %s\n Hash:%s"%(self.transactions, self.hash)
    


In [33]:
class BlockChain:
    """
        区块链结构体
            blocks:  包含的区块列表
    """
    
    def __init__(self):
        self.blocks = []
    
    def add_block(self, block):
        """
        添加区块
        """
        self.blocks.append(block)

In [34]:
#from mlxtend.preprocessing import TransactionEncoder
class TransactionEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__

In [35]:
#updated ProofOfWork version with a rewarding system:

class ProofOfWork:
    """
        工作量证明
    """
    
    def __init__(self, block, miner, difficult=5):
        self.block=block
        self.miner=miner
        
        #定义工作量难度，默认为5，表示有效的hash以5个0开头
        self.difficulty=difficult
        
        #添加工作奖励
        self.reward_amount=10
        
        
    def mine(self):
        """
            挖矿函数
        """
        i=0
        prefix='0'*self.difficulty
        
        #添加奖励
        t = Transaction(
            sender = "",
            recipient=self.miner.address,
            amount=self.reward_amount,
        )
        sig = self.miner.sign(json.dumps(t, indent=4, cls=TransactionEncoder))
        t.set_sign(sig, self.miner.pubkey)
        self.block.transactions.append(t)
        
        while True:
            message = hashlib.sha256()
            message.update(str(self.block.prev_hash).encode('utf-8'))
            message.update(str(self.block.transactions).encode('utf-8'))
            message.update(str(self.block.timestamp).encode('utf-8'))
            message.update(str(i).encode("utf-8"))
            digest=message.hexdigest()
            if digest.startswith(prefix):
                self.block.nonce=i
                self.block.hash=digest
                return self.block
            i+=1
        

In [36]:
def get_balance(user):
    balance = 0
    for block in blockchain.blocks:
        for t in block.transactions:
            if t.sender == user.address.decode():
                balance-=t.amount
            elif t.recipient == user.address.decode():
                balance+=t.amount
    return balance

In [37]:
#测试，初始化区块链
blockchain = BlockChain()

#创建三个钱包，alice, tom, bob
alice = Wallet()
tom = Wallet()
bob = Wallet()


#打印当前钱包情况
print("alice: %d number of coins"%(get_balance(alice)))
print("tom: %d number of coins"%(get_balance(tom)))
print("bob: %d number of coins"%(get_balance(bob)))

alice: 0 number of coins
tom: 0 number of coins
bob: 0 number of coins


In [38]:
#alice 生成创世区块，并添加到区块链中

new_block1 = Block(transactions=[], prev_hash="")
w1 = ProofOfWork(new_block1, alice)
genesis_block = w1.mine()
blockchain.add_block(genesis_block)

In [39]:
print("alice:%d numbers of coins"%(get_balance(alice)))

alice:10 numbers of coins


In [40]:
#alice转给 Tom 0.3个加密货币
transactions = []
new_transaction = Transaction(
    sender=alice.address,
    recipient=tom.address,
    amount=3.6)
sig=tom.sign(str(new_transaction))
new_transaction.set_sign(sig, tom.pubkey)

#打印当前钱包情况
print("alice: %d numbers of coins" %(get_balance(alice)))
print("tom: %d numbers of coins" %(get_balance(tom)))
print("bob: %d numbers of coins" %(get_balance(bob)))

alice: 10 numbers of coins
tom: 0 numbers of coins
bob: 0 numbers of coins


In [41]:
#bob在网络上接收到这笔交易信息，进行验证没问题后生成一个新的区块并添加到区块链中

if verify_sign(new_transaction.pubkey,
                str(new_transaction),
                  new_transaction.signature):
    
    #验证交易签名没问题，生成一个新的区块
    print("verifying success")
    new_block2 = Block(transactions=[new_transaction], prev_hash="")
    print("new block created...")
    w2 = ProofOfWork(new_block2, bob)
    block=w2.mine()
    print("adding new block to blockchain")
    blockchain.add_block(block)
else:
    print("failed transaction!")

verifying success
new block created...
adding new block to blockchain


In [42]:
#打印当前钱包情况
print("alice: %.1f numbers of coins" %(get_balance(alice)))
print("tom: %.1f numbers of coins" %(get_balance(tom)))
print("bob: %d numbers of coins" %(get_balance(bob)))


alice: 6.4 numbers of coins
tom: 3.6 numbers of coins
bob: 10 numbers of coins


In [43]:
print(alice.address)
print(tom.address)
print(bob.address)

b'WBcm9NEzJ0xWkYq2h3Cl+5Eyv4Lo3mBJQctXOANfnVc='
b'icCheCI3QnP+XhFG4zYP9a7FXh7onhyj5T1snYHstXE='
b'+ncPL31NzXeT2Jfo7hIrLEDnolpYMrZll3tBuFq0eys='


In [44]:
print("There are %d number of blocks in it\n"%len(blockchain.blocks))

for block in blockchain.blocks:
    print("previous hash: %s"%block.prev_hash)
    print("transaction: %s"%block.transactions)
    print("time stamp: %s"%block.timestamp)
    print("hash %s\n"%block.hash)

There are 2 number of blocks in it

previous hash: 
transaction: [WBcm9NEzJ0xWkYq2h3Cl+5Eyv4Lo3mBJQctXOANfnVc= getting 10 number of coins from mining]
time stamp: 2022-08-05 10:03:31
hash 00000f75280ea3f5fb6e756515e8edbe404315708c83e90f6887207896483b5a

previous hash: 
transaction: [WBcm9NEzJ0xWkYq2h3Cl+5Eyv4Lo3mBJQctXOANfnVc= sending 3 number of coins to icCheCI3QnP+XhFG4zYP9a7FXh7onhyj5T1snYHstXE=, +ncPL31NzXeT2Jfo7hIrLEDnolpYMrZll3tBuFq0eys= getting 10 number of coins from mining]
time stamp: 2022-08-05 10:03:36
hash 000001dc2d1c85d5be46b692f6688f97496795f5ed6c1c24a6c0bbc0c15ad91e

